In [ ]:
import yaml
import numpy as np
from matplotlib import pyplot as plt

# --- Custom Utility Module ---
from mineral_utils import Paleodetector, slice_spectrum, detection_model_efficiency

print("Libraries imported successfully.")

In [ ]:
mineral_name =  "Olivine"  # Must match a key in MINERAL_LIBRARY

sample_mass_kg = 0.000002       # Sample mass in kg (e.g., 0.01 for 10g)

x_bins = np.linspace(0, 50000, 201)        #Track length bins in nm
x_mids = x_bins[:-1] + np.diff(x_bins) / 2

In [ ]:
CONFIG = yaml.safe_load(open('Data/basic_config.yaml', 'r'))
MINERAL_LIBRARY = yaml.safe_load(open('Data/mineral_library.yaml', 'r'))

In [ ]:
exposure_times = np.array([41.4, 30., 13.1, 10.96, 9.5, 8.64, 7.65])
exposure_times_err = np.array([1., 2., 0.7, 0.15, 0.5, 0.06, 0.12])
names = np.array(["Laschamp", "Lemptegy", "Come", "Dome", "Pariou", "La Vache", "Montcineyre"], dtype=str)
energies = np.array(CONFIG['geant4_energy_bins_gev'])[:90]

In [ ]:
scenario_config_simple = {
    'name': 'simple',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
        40.: ('H3a', 'H3a'),
    }
}

scenario_config_simple_enhanced = {
    'name': 'simple_enhanced',
    'event_fluxes': {
        0.: ('H3a_enhanced', 'H3a_enhanced'),
        1.4: ('H3a_enhanced', 'H3a_enhanced'),
        6.4: ('H3a', 'H3a'),
        41.4: ('H3a', 'H3a'),
    }
}

scenario_config_SN250_enhanced = {
    'name': 'SN250_enhanced',
 'event_fluxes': {1.4: ('SN250pc_7kyr_enhanced', 'SN250pc_7kyr_enhanced'),
  3.9: ('SN250pc_10kyr_enhanced', 'SN250pc_10kyr_enhanced'),
  6.4: ('SN250pc_12kyr', 'SN250pc_12kyr'),
  8.9: ('SN250pc_15kyr', 'SN250pc_15kyr'),
  11.4: ('SN250pc_17kyr', 'SN250pc_17kyr'),
  13.9: ('SN250pc_20kyr', 'SN250pc_20kyr'),
  16.4: ('SN250pc_22kyr', 'SN250pc_22kyr'),
  18.9: ('SN250pc_25kyr', 'SN250pc_25kyr'),
  21.4: ('SN250pc_27kyr', 'SN250pc_27kyr'),
  23.9: ('SN250pc_30kyr', 'SN250pc_30kyr'),
  26.4: ('SN250pc_32kyr', 'SN250pc_32kyr'),
  28.9: ('SN250pc_35kyr', 'SN250pc_35kyr'),
  31.4: ('SN250pc_37kyr', 'SN250pc_37kyr'),
  33.9: ('SN250pc_40kyr', 'SN250pc_40kyr'),
  36.4: ('SN250pc_42kyr', 'SN250pc_42kyr'),
  38.9: ('SN250pc_45kyr', 'SN250pc_45kyr'),
  41.4: ('SN250pc_47kyr', 'SN250pc_47kyr')}
}

keeys = np.linspace(0, np.floor(exposure_times[0]/2.5)*2.5, int(np.round(exposure_times[0]/2.5)))
vaals = np.floor(7.5 + np.linspace(0, np.floor(exposure_times[0]/2.5)*2.5, int(np.round(exposure_times[0]/2.5))))
scenario_config_SN250 = []
for i in range(len(exposure_times)):
    keys = np.round(keeys[keeys<exposure_times[i]]+(exposure_times[i]%2.5), 2)
    values = [(f'SN250pc_{int(val)}kyr', f'SN250pc_{int(val)}kyr') for val in vaals[-len(keeys[keeys<exposure_times[i]]):]]

    d = dict(zip(keys, values))
    dd = {'name':f'SN250_{exposure_times[i]}', 'event_fluxes':d}
    scenario_config_SN250.append(dd)

In [ ]:
# --- Setup & Verification ---
mineral_config = MINERAL_LIBRARY.get(mineral_name)
if not mineral_config:
    raise ValueError(f"Mineral '{mineral_name}' not found in MINERAL_LIBRARY.")

mineral = Paleodetector(mineral_config)

In [ ]:
total_fission_tracks = np.array([mineral.integrate_fission_spectrum(x_bins, age, sample_mass_kg) for age in exposure_times])

In [ ]:
total_r_neutron_tracks = np.array([mineral.integrate_background_neutron_spectrum(x_bins, energies, age, sample_mass_kg) for age in exposure_times])

In [ ]:
total_cosmo_tracks_simple = [mineral.integrate_all_particles(x_bins, scenario_config=scenario_config_simple, energy_bins_gev=energies, exposure_window_kyr=age, initial_depth=0.05, sample_mass_kg = sample_mass_kg, steps=3) for age in exposure_times]

In [ ]:
total_cosmo_tracks_SN250 = [mineral.integrate_all_particles(x_bins, scenario_config=scenario_config_SN250[i], energy_bins_gev=energies, exposure_window_kyr=age, initial_depth=0.05, sample_mass_kg = sample_mass_kg, steps=int(age/2)) for i, age in enumerate(exposure_times)]

In [ ]:
tracks_enhanced_simple = mineral.integrate_all_particles(x_bins, scenario_config=scenario_config_simple_enhanced, energy_bins_gev=energies, exposure_window_kyr=exposure_times[0], initial_depth=0.05, sample_mass_kg = sample_mass_kg, steps=20)
tracks_enhanced_SN250 = mineral.integrate_all_particles(x_bins, scenario_config=scenario_config_SN250_enhanced, energy_bins_gev=energies, exposure_window_kyr=exposure_times[0], initial_depth=0.05, sample_mass_kg = sample_mass_kg, steps=20)

In [ ]:
i = -1

fig, ax = plt.subplots(figsize=(10, 6))
ax.step(x_mids/1000., total_fission_tracks[i], label='Fission Tracks', color='blue', where='mid')
ax.step(x_mids/1000., total_r_neutron_tracks[i], label='Background Neutrons', color='orange', where='mid')
ax.step(x_mids/1000., total_cosmo_tracks_simple[i]['total'], label='Cosmogenic (Simple)', color='green', where='mid')
ax.step(x_mids/1000., total_cosmo_tracks_SN250[i]['total'], label='Cosmogenic (SN250)', color='red', where='mid')
#ax.step(x_mids/1000., tracks_enhanced_simple['total'], label='Cosmogenic (Simple Enhanced)', color='cyan', linestyle='--', where='mid')
#ax.step(x_mids/1000., tracks_enhanced_SN250['total'], label='Cosmogenic (SN250 Enhanced)', color='magenta', linestyle='--', where='mid')

ax.set_yscale('log')
ax.set_xlim(0, 25.)
ax.set_ylim(1e-2, 1e4)
ax.set_xlabel(r'Track Length (m)')
ax.set_ylabel('Track Count')
ax.set_title(f'Track Length Distributions for {mineral_name} at {exposure_times[i]} kyr Exposure')
ax.legend()
plt.show()

In [ ]:
sliced_simple = slice_spectrum(x_bins, total_cosmo_tracks_simple[i]['total']+total_fission_tracks[i]+total_r_neutron_tracks[i], pit_width=1500.)
sliced_SN250 = slice_spectrum(x_bins, total_cosmo_tracks_SN250[i]['total']+total_fission_tracks[i]+total_r_neutron_tracks[i], pit_width=1500.)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.step(x_mids/1000., sliced_simple, label='Total tracks in simple scenario', color='blue', where='mid')
ax.step(x_mids/1000., sliced_SN250, label='Total tracks in SN250 scenario', color='orange', where='mid')

ax.set_yscale('log')
ax.set_xlim(0, 20.)
ax.set_ylim(5e-1, 1e2)
ax.set_xlabel(r'Track Length (m)')
ax.set_ylabel('Track Count')
ax.set_title(f'Track Length Distributions for {mineral_name} at {exposure_times[i]} kyr Exposure')
ax.legend()
plt.show()